In [ ]:
%load_ext autoreload
%autoreload 2

#pyplot
import matplotlib.pyplot as plt

# model
import torch
import torch_geometric


from auxiliary.config import get_config, ModelType, NetworkType
from training.models import GCN, FCNN
from training.dataloader import DataLoader
from training.training import Run, evaluate_dataset
import training.evaluation as metrics
from training.postprocessing import ActSpaceCache

In [ ]:
config = get_config()
train_config = config['training']

# Loading model
Note: config must contain the hyperparameters used by the model

In [ ]:
model_type = config['training']['hyperparams']['model_type']

if model_type == ModelType.GCN:
    model = GCN(train_config['hyperparams']['LReLu_neg_slope'],
                train_config['hyperparams']['weight_init_std'],
                train_config['GCN']['constants']['N_f_gen'],
                train_config['GCN']['constants']['N_f_load'],
                train_config['GCN']['constants']['N_f_endpoint'],
                train_config['GCN']['hyperparams']['N_GCN_layers'],
                train_config['hyperparams']['N_node_hidden'],
                train_config['GCN']['hyperparams']['aggr'],
                train_config['GCN']['hyperparams']['network_type'],
                train_config['GCN']['hyperparams']['layer_type'],
                train_config['GCN']['hyperparams']['GINConv_nn_depth'])
elif model_type == ModelType.FCNN:
    model = FCNN(train_config['hyperparams']['LReLu_neg_slope'],
                 train_config['hyperparams']['weight_init_std'],
                 train_config['FCNN']['constants']['size_in'],
                 train_config['FCNN']['constants']['size_out'],
                 train_config['FCNN']['hyperparams']['N_layers'],
                 train_config['hyperparams']['N_node_hidden'])

In [ ]:
model_path = config['paths']['model']
state_dict = torch.load(model_path)
model.load_state_dict(state_dict)

# Analysing seen data

In [ ]:
seen_processed_data_path = config['paths']['data']['processed']
seen_matrix_cache_path = seen_processed_data_path + 'auxiliary_data_objects/con_matrix_cache.json'

unseen_processed_data_path = 'data/processed/greedy_unseen/'
unseen_matrix_cache_path = unseen_processed_data_path + 'auxiliary_data_objects/con_matrix_cache.json'

feature_statistics_path = seen_processed_data_path + 'auxiliary_data_objects/feature_stats.json'
model_type = config['training']['hyperparams']['model_type']
network_type = train_config['GCN']['hyperparams']['network_type']

dataloaders = {'seen': {}, 'unseen': {}}
for data, processed_data_path, matrix_cache_path in [('seen', seen_processed_data_path, seen_matrix_cache_path)]:#: , #('unseen', unseen_processed_data_path, unseen_matrix_cache_path)):
    for partition in ('train', 'val', 'test'):
        dataloaders[data][partition] = DataLoader(processed_data_path + '/' + partition, matrix_cache_path, feature_statistics_path, 'cpu',  model_type, network_type)

In [ ]:
%%capture

# Initialize metrics objects
IA = metrics.IncrementalAverage
metrics_dict = {
    'macro_accuracy': (metrics.macro_accuracy, IA()),
    'micro_accuracy': (metrics.micro_accuracy, IA()),
    'n_predicted_changes': (metrics.n_predicted_changes, IA()),
    'any_predicted_changes': (metrics.any_predicted_changes, IA()),
    'macro_accuracy_one_sub': (metrics.macro_accuracy_one_sub, IA()),
    'micro_accuracy_one_sub': (metrics.micro_accuracy_one_sub, IA()),
    'train_loss': (lambda **kwargs: kwargs['l'], IA()),
    'accuracy_predicted_substation':
        (metrics.accuracy_predicted_substation, IA())
}
val_metrics_dict = dict([('val_' + k, v) for k, v in metrics_dict.items()])
val_metrics_dict['val_macro_accuracy_valid'] = (metrics.macro_accuracy_valid, IA())
val_metrics_dict['val_micro_accuracy_valid'] = (metrics.micro_accuracy_valid, IA())
IAM = metrics.IncrementalAverageMetrics
eval_metrics = IAM(val_metrics_dict)

# Initialize action space cache used for
as_cache = ActSpaceCache(line_outages_considered=train_config['settings']['line_outages_considered'])

# Analyzing seen topologies

In [ ]:
%%capture
%tb
plt.ioff()
results = {}

for partition, dataloader in dataloaders['seen'].items():
        
    eval_metrics.reset()
    _ = evaluate_dataset(model, dataloader, as_cache, eval_metrics)

    results[partition] = {
        'val_macro_accuracy_valid':  eval_metrics.metrics_dict['val_macro_accuracy_valid'][1].get(),
        'loss':  eval_metrics.metrics_dict['val_train_loss'][1].get()
    }


In [ ]:
results

# Analyzing unseen topologies

In [ ]:
%%capture
for data, processed_data_path, matrix_cache_path in [('unseen', unseen_processed_data_path, unseen_matrix_cache_path)]:
    for partition in ('train', 'val', 'test'):
        dataloaders[data][partition] = DataLoader(processed_data_path + '/' + partition, matrix_cache_path, feature_statistics_path, 'cpu',  model_type, network_type)

# Initialize metrics objects
IA = metrics.IncrementalAverage
metrics_dict = {
    'macro_accuracy': (metrics.macro_accuracy, IA()),
    'micro_accuracy': (metrics.micro_accuracy, IA()),
    'n_predicted_changes': (metrics.n_predicted_changes, IA()),
    'any_predicted_changes': (metrics.any_predicted_changes, IA()),
    'macro_accuracy_one_sub': (metrics.macro_accuracy_one_sub, IA()),
    'micro_accuracy_one_sub': (metrics.micro_accuracy_one_sub, IA()),
    'train_loss': (lambda **kwargs: kwargs['l'], IA()),
    'accuracy_predicted_substation':
        (metrics.accuracy_predicted_substation, IA())
}
val_metrics_dict = dict([('val_' + k, v) for k, v in metrics_dict.items()])
val_metrics_dict['val_macro_accuracy_valid'] = (metrics.macro_accuracy_valid, IA())
val_metrics_dict['val_micro_accuracy_valid'] = (metrics.micro_accuracy_valid, IA())
IAM = metrics.IncrementalAverageMetrics
eval_metrics = IAM(val_metrics_dict)

# Initialize action space cache used for
as_cache = ActSpaceCache(line_outages_considered=train_config['settings']['line_outages_considered'])
#_ = evaluate_dataset(model, dataloader, as_cache, eval_metrics)

In [ ]:
%%capture
%tb
plt.ioff()
results = {}

for partition, dataloader in dataloaders['unseen'].items():
        
    eval_metrics.reset()
    figures = evaluate_dataset(model, dataloader, as_cache, eval_metrics)

    results[partition] = {
        'val_macro_accuracy_valid':  eval_metrics.metrics_dict['val_macro_accuracy_valid'][1].get(),
        'loss':  eval_metrics.metrics_dict['val_train_loss'][1].get()
    }


In [ ]:
results

In [ ]:
figures['substation_confusion_matrix']

In [ ]:
figures['affected_objects']

In [ ]:
figures['accuracy_per_lineout']

In [ ]:
figures['accuracy_per_label']